In [1]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict

/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


# Avslutte manuelt innlagt i eldre emne

## Problemstilling
Når vi legg inn brukarar manuelt i emne vert dei liggande aktive etter at emneet er avslutta, for det vert ikkje sendt melding om å avslutte innmeldinga for desse, slik det vert gjort for dei som kjem frå FS.

## Løysing
Eg reknar alle emne som har `enrollment_term_id` i lista over `avslutt_semestre` som avslutta. Det er desse som får "avslutt"-melding frå FS våren 2025 (kanskje). 
Så går eg gjennom denne lista (ca. 16000 emne) og finn alle personar som framleis er aktive. 

## Løysing 1: Hente alle emne
Først hentar eg alle emna vi har i Canvas.

In [2]:
url = "https://hvl.instructure.com/api/v1/accounts/1/courses"

dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
        dataliste = []
        for element in data:
            dataliste.append([element['id'], element['name'], element['enrollment_term_id']])
        df = pd.DataFrame(dataliste, columns=['id', 'name', 'enrollment_term_id'])
        dr_liste.append(df)
# og etter at eg er ferdig å lese inn slår eg dei same:
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)


https://hvl.instructure.com/api/v1/accounts/1/courses?page=2&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=3&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=4&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=5&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=6&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=7&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=8&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=9&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=10&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=11&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=12&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=13&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=14&per_page=100
https://hvl.instructure.com/api/v

Så vasker eg data med Data Wrangler, og får ein funksjon som eg kan køyre på datasettet før eg lagrar det til feil:

In [6]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(alledata):
    # Drop column: 'name'
    alledata = alledata.drop(columns=['name'])
    return alledata

alledata_clean = clean_data(alledata.copy())
alledata_clean.head()

,id,enrollment_term_id
0,2,1
1,5,1
2,10,1
3,11,2
4,12,1


In [7]:
alledata_clean.to_csv("alle_emne_250624.csv", index=False)